<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/mobile-net/Algorithums/pretrained/MobileNetV3/MobileNetV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNet v3

## Imports

### Collab Imports

In [1]:
from google.colab import drive

### Default Nessary Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### Tensorflow Imports

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense,Conv2D, MaxPooling2D, Flatten,GlobalAveragePooling2D

## Variables

### Path variables

In [11]:
dataset_dir='/content/drive/MyDrive/AYUSHVISION/working' #datasetbase dirctory

In [18]:
dataset_train_dir=os.path.join(dataset_dir,'train')
dataset_test_dir=os.path.join(dataset_dir,'val')
dataset_val_dir=os.path.join(dataset_dir,'val')

### Genrator Variable

In [6]:
rescale=1/255

### Configration Variables

In [7]:
target_size=(224,224)
class_mode='categorical'
input_shape=(224,224,3)

### Model Variables

#### Number If Classes

In [32]:
no_of_classes=1

## Envirumentatl Setup

### Collab

In [9]:
drive.mount('/content/drive') # drive activate for Use

Mounted at /content/drive


## Validations

In [19]:
f'{dataset_train_dir} is Correct' if os.path.exists(dataset_train_dir) and os.path.isdir(dataset_train_dir) else f' {dataset_train_dir} is incorrect'


'/content/drive/MyDrive/AYUSHVISION/working/train is Correct'

In [20]:
f'{dataset_test_dir} is Correct' if os.path.exists(dataset_test_dir) and os.path.isdir(dataset_train_dir) else f' {dataset_test_dir} is incorrect'


'/content/drive/MyDrive/AYUSHVISION/working/val is Correct'

In [21]:
f'{dataset_val_dir} is Correct' if os.path.exists(dataset_val_dir) and os.path.isdir(dataset_val_dir) else f' {dataset_val_dir} is incorrect'

'/content/drive/MyDrive/AYUSHVISION/working/val is Correct'

## DataSet

### Image Genarator

In [14]:
train_data_genrator=image.ImageDataGenerator(rescale=rescale)

In [15]:
test_data_genrator=image.ImageDataGenerator(rescale=rescale)

In [16]:
val_data_genrator=image.ImageDataGenerator(rescale=rescale)

### Prepared Dataset

In [22]:
train_ds_gen=train_data_genrator.flow_from_directory(
    dataset_train_dir,
    target_size=target_size,
    class_mode=class_mode
)
val_ds_gen=val_data_genrator.flow_from_directory(
    dataset_val_dir,
    target_size=target_size,
    class_mode=class_mode
)

test_ds_gen=test_data_genrator.flow_from_directory(
    dataset_test_dir,
    target_size=target_size,
    class_mode=class_mode
)

Found 140 images belonging to 1 classes.
Found 20 images belonging to 1 classes.
Found 20 images belonging to 1 classes.


## Model


### Base Model MobileNetV3

In [33]:
ayush_mobilenetV3_base=MobileNetV3Large(input_shape=input_shape,  # We don't want the dense layers on top
                       weights=None)

### Layers setup for Base Model

In [34]:
ayush_mobilenetV3_base.trainable=True

### Final Model

In [35]:
ayushVision_MobileNetV3_Model=Sequential([ayush_mobilenetV3_base])


#### Middle Layers

#### Last Layer

In [36]:
ayushVision_MobileNetV3_Model.add(Dense(no_of_classes,activation='softmax'))

## Model Compile

#### Optimizer

In [27]:
optimizer = Adam(learning_rate=0.001)

#### Compile

##### Variables

In [37]:
metrics=["accuracy"]
lossfn='categorical_crossentropy'

In [38]:
ayushVision_MobileNetV3_Model.compile(
    optimizer=optimizer,
    loss=lossfn,
    metrics=metrics)

### Trainning

In [39]:
epochs=10

In [40]:
model_history=ayushVision_MobileNetV3_Model.fit(
    train_ds_gen,
    epochs=epochs,
    validation_data=val_ds_gen)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored

5/5 ━━━━━━━━━━━━━━━━━━━━ 113s 13s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 112ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 1.0000 - los

### Evaluation

In [41]:
test_loss, test_accuracy = ayushVision_MobileNetV3_Model.evaluate(test_ds_gen)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 1.0000 - loss: 0.0000e+00


In [42]:
test_accuracy

1.0